In [2]:
# Private HTTP Endpoint
# Pattern follows 'http://<endpoint-name>-<commit-id>.deployment.svc.cluster.local:16030'
MODEL_ENDPOINT = 'http://long-bread-5258-54d99e03.deployment.svc.cluster.local:16030'
MODEL_METRICS_ENDPOINT = MODEL_ENDPOINT + "/metrics"
MODEL_INFER_ENDPOINT = MODEL_ENDPOINT + "/infer"

# Endpoint Ping using Pycurl

In [3]:
import pycurl
from io import BytesIO 
from IPython.core.display import display, HTML

def ping_homepage(url):

    b_obj = BytesIO() 
    crl = pycurl.Curl() 

    # Set URL value
    crl.setopt(crl.URL, url)
    # Write bytes that are utf-8 encoded
    crl.setopt(crl.WRITEDATA, b_obj)
    # Perform the request
    crl.perform() 
    # End curl session
    crl.close()
    # Get the content stored in the BytesIO object (in byte characters) 
    get_body = b_obj.getvalue()
    # Decode the bytes stored in get_body to HTML and print the result 
    # print('Output of GET request:\n%s' % get_body.decode('utf8')) 
    display(HTML(get_body.decode('utf8')))
    
ping_homepage(MODEL_ENDPOINT)

# Testing the model inference

In [74]:
import requests
import pandas as pd

In [75]:
df = pd.read_csv("creditdata_test_v2.csv")

In [76]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,TARGET
0,1559,50000.0,2,3,1,31,1,2,2,2,...,40906.0,42100.0,41269.0,2830.0,1000.0,1800.0,2000.0,0.0,1704.0,0
1,16879,500000.0,1,2,1,38,0,0,0,0,...,49515.0,42568.0,48268.0,30107.0,10026.0,7241.0,50091.0,25000.0,5078.0,1
2,29295,50000.0,1,2,2,24,0,0,0,0,...,17827.0,7655.0,7881.0,5000.0,1700.0,2000.0,500.0,500.0,500.0,1
3,18442,200000.0,2,2,1,25,0,0,0,0,...,110828.0,110592.0,103734.0,5000.0,4200.0,3820.0,4000.0,3760.0,4006.0,0
4,23457,270000.0,2,1,2,37,-2,-2,-2,-2,...,1350.0,558.0,0.0,6380.0,16098.0,1350.0,558.0,0.0,128934.0,1


In [77]:
FEATURES = [
#     'ID',
    'LIMIT_BAL',
    'SEX',
    'EDUCATION',
    'MARRIAGE',
    'AGE',
    'PAY_1',
    'PAY_2',
    'PAY_3',
    'PAY_4',
    'PAY_5',
    'PAY_6',
    'BILL_AMT1',
    'BILL_AMT2',
    'BILL_AMT3',
    'BILL_AMT4',
    'BILL_AMT5',
    'BILL_AMT6',
    'PAY_AMT1',
    'PAY_AMT2',
    'PAY_AMT3',
    'PAY_AMT4',
    'PAY_AMT5',
    'PAY_AMT6'
]

In [78]:
def get_row_of_data(df, row_index=0):
    row = df.iloc[row_index]    
    data = {}
    for feat in FEATURES:
        data[feat] = row[feat]
        
    print("TARGET == {}".format(row["TARGET"]))

    return data

# data = get_row_of_data(df, 1)
# data

In [101]:
# Model Deployment: Endpoint URL

# Prepare data
data = get_row_of_data(df, 0)

headers = {
    'Content-Type': 'application/json'
}

response = requests.post(MODEL_INFER_ENDPOINT, headers=headers, data=json.dumps(data))
print(response.ok)
print(response.json())

TARGET == 0.0
True
{'inference': 1.0}


In [88]:
# response.content

In [64]:
# response.text